# ベクトルストアをリトリーバーとして使用する方法

ベクトルストアリトリーバーは、 [retriever](/docs/concepts/retrievers/) として [vector store](/docs/concepts/vectorstores/) を使用するものです。  
これは、リトリーバーの [interface](/docs/concepts/runnables/)に適合するように、ベクトルストアクラスを軽量にラップしたものです。  
ベクトルストアに実装された検索メソッド（類似性検索やMMRなど）を利用して、ベクトルストア内のテキストをクエリします。

このガイドでは、以下の内容を取り上げます：

1. ベクトルストアからリトリーバーをインスタンス化する方法
2. リトリーバーの検索タイプを指定する方法
3. スコアの閾値やTop-kなどの追加検索パラメータを指定する方法

## ベクトルストアからリトリーバーを作成する

ベクトルストアからリトリーバーを作成するには、ベクトルストアの[.as_retriever](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html#langchain_core.vectorstores.base.VectorStore.as_retriever) メソッドを使用します。  
以下で具体例を見ていきましょう。

まず、ベクトルストアをインスタンス化します。  
ここでは、メモリ内の [FAISS](https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.faiss.FAISS.html) ベクトルストアを使用します。

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("state_of_the_union.txt")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)

retrieverをインスタンス化する

In [2]:
retriever = vectorstore.as_retriever()

これにより、retriever（具体的には[VectorStoreRetriever](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html)）が作成されます。このretrieverは通常の方法で使用できます。

In [3]:
docs = retriever.invoke("what did the president say about ketanji brown jackson?")

## 最大限のマージナル関連性（Maximum Marginal Relevance）による検索
デフォルトでは、ベクトルストアリトリーバーは類似性検索を使用します。  
ただし、ベースとなるベクトルストアが最大限のマージナル関連性（Maximum Marginal Relevance, MMR）検索をサポートしている場合、検索タイプとしてそれを指定することができます。

これにより、基盤となるベクトルストアで使用される具体的なメソッド（例: `similarity_search`、`max_marginal_relevance_search` など）を明示的に指定することが可能です。

In [4]:
retriever = vectorstore.as_retriever(search_type="mmr")

In [5]:
docs = retriever.invoke("what did the president say about ketanji brown jackson?")

## 検索パラメータの指定

`search_kwargs` を使用して、基盤となるベクトルストアの検索メソッドにパラメータを渡すことができます。

### 類似スコアの閾値を設定した検索

例えば、類似スコアの閾値を設定し、その閾値を超えるスコアを持つドキュメントのみを返すように指定できます。

In [6]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)

In [7]:
docs = retriever.invoke("what did the president say about ketanji brown jackson?")

### 上位k件の指定

リトリーバーが返すドキュメントの数を `k` で制限することもできます。

In [8]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [9]:
docs = retriever.invoke("what did the president say about ketanji brown jackson?")
len(docs)

1